## Download all option expirations for a given ticker

In [232]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup as BS
import datetime
import pandas as pd
import numpy as np

In [233]:
OPTIONS_API_URL = "https://query2.finance.yahoo.com/v7/finance/options/"
OPTIONS_WEB_URL = "https://finance.yahoo.com/quote/"

ticker = "SPY"
price = 276.0

def get_list_dates(ticker):
    # get expiration dates for a given ticker
    url = OPTIONS_API_URL + ticker
    usock = urlopen(url)
    data = usock.read()
    usock.close()
    soup = BS(data,"lxml")
    python_dict = json.loads(soup.getText())
    list_dates=python_dict['optionChain']['result'][0]['expirationDates']
    return list_dates

def get_options(ticker):
    # get all expiration option contracts for a given ticker
    call = pd.DataFrame()
    put = pd.DataFrame()
    name1 = name.replace('WIKI/', '')
    list_dates = get_list_dates(ticker)

    for j in list_dates:
        try:
            calls = pd.read_html(OPTIONS_WEB_URL + ticker + "/options?p=" + ticker + "&date=" + str(j))[0]
            calls["expiration"] = datetime.datetime.fromtimestamp(int(j)).strftime('%Y-%m-%d')
            call = call.append(calls)
        except Exception as err: print("Error: {0}".format(err))
        try:
            puts = pd.read_html(OPTIONS_WEB_URL + ticker + "/options?p=" + ticker + "&date=" + str(j))[1]
            puts["expiration"] = datetime.datetime.fromtimestamp(int(j)).strftime('%Y-%m-%d')
            put = put.append(puts)
        except Exception as err: print("Error: {0}".format(err))
            
    call["type"], put["type"] = "call", "put"
    
    final = call.append(put)
    final["ticker"] = ticker
    
    return final

In [234]:
options_res = get_options(ticker)

Error: list index out of range


#### Clean up tasks
1. Add a column for ITM, LT_Day, LT_Hour, LT_Min
2. Remove - from % change column
3. Clean up data for last trade, remove EDT, add a space before PM and AM

In [244]:
options_res.columns

Index(['Contract Name', 'Last Trade Date', 'Strike', 'Last Price', 'Bid',
       'Ask', 'Change', '% Change', 'Volume', 'Open Interest',
       'Implied Volatility', 'expiration', 'type', 'ticker'],
      dtype='object')

In [245]:
options_clean = options_res.copy()
options_clean.reset_index(drop=True, inplace=True)

In [246]:
# convert strings to numbers
options_clean['% Change'] = options_clean['% Change'].str.replace('%','').replace('-',0).astype(float) / 100
options_clean['Implied Volatility'] = options_clean['Implied Volatility'].str.replace('%','').replace('-',0).astype(float) / 100

# convert to dates
options_clean['Last Trade Date'] = pd.to_datetime(options_clean['Last Trade Date'])
options_clean['expiration'] = pd.to_datetime(options_clean['expiration'])

# ITM filter
options_clean['ITM'] = 0
options_clean.loc[options_clean[(options_clean['type'] == 'call') & (options_clean['Strike'] < price)].index, 'ITM'] = 1
options_clean.loc[options_clean[(options_clean['type'] == 'put') & (options_clean['Strike'] > price)].index, 'ITM'] = 1

# 0 for Monday and 6 for Sunday
options_clean['LT_WeekDay'], options_clean['LT_Hour'] = \
options_clean['Last Trade Date'].dt.dayofweek, \
options_clean['Last Trade Date'].dt.hour

# calcuates Bid/Ask spreads
options_clean['B/A Spread'] = options_clean['Ask'] - options_clean['Bid']
options_clean['B/A Spread %'] = options_clean['B/A Spread'] / options_clean['Bid']

/home/ubuntu/.local/lib/python3.5/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [247]:
calls_itm, puts_itm = options_clean[(options_clean['type'] == 'call') & (options_clean['Strike'] < price)].Strike.count(), \
options_clean[(options_clean['type'] == 'put') & (options_clean['Strike'] > price)].Strike.count()
total, ITM = options_res.Strike.count(), options_clean[options_clean.ITM == 1].ITM.count()
assert ITM == calls_itm + puts_itm

In [248]:
options_clean.head()

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,expiration,type,ticker,ITM,LT_WeekDay,LT_Hour,B/A Spread,B/A Spread %
0,SPY180711C00240000,2018-06-08 16:05:00,240.0,37.24,38.19,38.41,0.0,0.0,2,0,2.1445,2018-07-11,call,SBUX,1,4,16,0.22,0.005761
1,SPY180711C00245000,2018-07-10 15:59:00,245.0,33.93,31.45,31.94,0.0,0.0,7,0,0.9531,2018-07-11,call,SBUX,1,1,15,0.49,0.015580
2,SPY180711C00250000,2018-07-09 10:47:00,250.0,27.01,25.50,27.96,0.0,0.0,5,5,1.5469,2018-07-11,call,SBUX,1,0,10,2.46,0.096471
3,SPY180711C00255000,2018-06-29 23:46:00,255.0,14.62,16.62,16.89,0.0,0.0,1,1,0.0000,2018-07-11,call,SBUX,1,4,23,0.27,0.016245
4,SPY180711C00259000,2018-06-27 14:37:00,259.0,13.81,12.62,12.81,0.0,0.0,1,0,0.0000,2018-07-11,call,SBUX,1,2,14,0.19,0.015055


In [240]:
options_clean.to_csv(ticker + '_options.csv')